# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [1]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

#진행률 바 
if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Get:13 https://developer.download.nv

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [3]:
def get_petition(wd, number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        #ID가 petitionsView_title인 element가 로딩될 때 까지 2초 대기
        element = WebDriverWait(wd, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        #lxml HTML 파서를 사용, html.parser보다 유연하고 빠른처리
        bs = BeautifulSoup(wd.page_source, 'lxml')

        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))
        if progress == '답변완료':
            contents = bs.find_all('div',{'class':'View_write'})[1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''
        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}

In [4]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    wd = webdriver.Chrome('chromedriver',options=options)
    print(get_petition(wd, 603234))
    print(get_petition(wd, 602925))
    wd.quit()

{'number': 603234, 'progress': '청원진행중', 'title': "JTBC 드라마 '설**'를 둘러싼 수많은 날조와 왜곡에 대한 진실", 'like': 4603, 'contents': '우선 이번 설** 사태는 드라마의 앞뒤 문맥을 고려하지 않은 채 특정 부분만을 단편적으로 편집하여 확대 해석한 내용들과 가짜 뉴스들이 온라인상에 급속도로 퍼지게 되면서 드라마가 전하고자 하는 진짜 의미와 의도에 대해 오해가 발생하여 빚어진 사태입니다.\n\n\n\n1. 작중에서 남한으로 간첩을 불러들인 곳은 다름 아닌 \'안기부\'입니다.\n\n\n남북한 수뇌부가 각각 정권 연장과 돈을 목적으로 야합하여 남파 공작원이 등장하게 된 것입니다.\n\n\n안기부는 간첩을 잡는 기관이라면서 본인들의 이익을 위해 진짜 간첩은 남한으로 불러들이고 죄 없는 학생들은 간첩몰이로 잡아갑니다.\n\n\n이것은 설**라는 드라마가 안기부 미화가 아닌 안기부의 모순을 꼬집고 있다고 볼 수 있는 것입니다.\n\n\n\n\n2. 설**는 중국 자본을 받은 드라마가 아닙니다.\n\n\n디즈니 플러스의 자본을 받은 드라마이고 jtbc가 중국 자본을 유치(2020.12.29.에 텐센트에서 투자) 받기 전 이미 설**는 11월에 촬영을 시작하여 jtbc가 자본을 투자 받기 전부터 촬영을 진행하고 있었습니다.\n\n\n텐센트는 의결권이 없는 상환 우환주 지분 투자라서 jtbc studio에 대한 경영권이 없습니다.\n\n\n그리고\xa0설**는 중국어로 설적화와 설화연이며 설**는 우리말입니다. 한자로는 雪**(눈 설, ***, **) 즉, 설**는 한국에서 사용하는 단어인 것입니다.\n\n\n\n\n\n3. 작중 1~2화에서 노래 \'솔아 솔아 푸르른 솔아\'는 남자 주인공인 수호가 안기부의 추격에서 도주하는 과정에서 잠깐 등장한 시위대가 부르던 노래입니다. 여기서 남주인 수호가 안기부의 추격에서 도망치면서 나오는 배경음악은 따로 존재하며, \'솔아 솔아 푸르른 솔아\'는 수호가 시위(

## 사용예

In [ ]:
import pandas as pd


def get_p(num = 603363, cnt = 4580):
    rtn = []
    wd = webdriver.Chrome('chromedriver',options=options)
    for idx, i in enumerate(tqdm(range(num-cnt, num))):
        tmp = get_petition(wd, i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
        # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
        if idx % 200 == 0:
            wd.quit()
            wd = webdriver.Chrome('chromedriver',options=options)
    return (rtn_c, rtn)


c, rtn = get_p()
%time wd = webdriver.Chrome('chromedriver',options=options)
# %time rtn.append(list(get_petition(wd, 582349).values()))
# %time rtn.append(list(get_petition(wd, 10).values()))
wd.quit()
pd.DataFrame(rtn, columns=c)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/4580 [00:00<?, ?it/s]

SessionNotCreatedException: ignored

[https://www1.president.go.kr/petitions/10](https://www1.president.go.kr/petitions/10)
- 관리자에 의해 비공개된 청원입니다

In [ ]:
df = pd.DataFrame(rtn, columns=c)

In [ ]:
df.to_csv('./petition_c.csv', encoding='utf-8', mode='w', index=False)

print('Completed..')